In [2]:
import os
import re
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit

In [3]:
import sys
sys.path.append("/g/g92/dskoda/prj/dskrc/python/inits")

from plotting import *
plt.rcParams['figure.dpi'] = 150
plt.style.use("paper")

In [4]:
hpr = pd.read_csv("02_data/nequip_hyperparams.csv", index_col=0)
lcv = pd.read_csv("02_data/nequip_learning.csv", index_col=0)
lls = pd.read_csv("02_data/nequip_losslands.csv", index_col=0)
sdf = pd.read_csv("02_data/nequip_entropy.csv", index_col=0)
exp = pd.read_csv("02_data/nequip_explosion.csv", index_col=0)
ext = pd.read_csv("02_data/nequip_extrapolation_errors.csv", index_col=0)

In [5]:
LABELS = {
    "baseline": "no rescaling",
    "non_trainable_bessel": "rescaling",
    "rescaling": "rescaling + bessel",
}

FIGS_DIR = "04_si"

In [6]:
MODELS = {
    "BIG": ["baseline", "non_trainable_bessel", "rescaling"],
    "2-OPT":  ["2-layer, baseline", "2-layer, AMSGrad-only", "2-layer, EMA-only"],
    "5-OPT":  ["5-layer, baseline", "5-layer, AMSGrad-only", "5-layer, EMA-only"],
    "LAYERS": ["2-layer", "3-layer", "4-layer", "5-layer"],
}

MODELS_ORDER = MODELS["BIG"] + MODELS["2-OPT"] + MODELS["5-OPT"] + MODELS["LAYERS"]

In [7]:
CMAPS = {
    "BIG": cm.Reds_r,
    "2-OPT": cm.Blues_r,
    "5-OPT": cm.Greens_r,
    "LAYERS": cm.Purples_r,
}

In [8]:
_colors = {}
for mset, cases in MODELS.items():
    norm = plt.Normalize(0, len(cases))
    for i, name in enumerate(cases):
        _colors[name] = CMAPS[mset](norm(i))
    
labels = pd.Series(_colors)

## Table with the hyperparameters

In [9]:
def is_trainable(builders):
    return "PerSpeciesRescale" in builders

hpr["rescaled"] = hpr["model_builders"].apply(is_trainable)

In [10]:
HPR_COLS = ["num_layers", "optimizer_amsgrad", "use_ema", 'BesselBasis_trainable', "rescaled"]

In [37]:
def df_to_latex_table(df, caption="", **kwargs):
    latex_str = df.to_latex(index=False, escape=False, **kwargs)
    latex_str = latex_str.replace('\\toprule', '\\hline \\hline')
    latex_str = latex_str.replace('\\midrule', '\\hline')
    latex_str = latex_str.replace('\\bottomrule', '\\hline \\hline')
    latex_str = latex_str.replace("True", r'\TrueMarker')
    latex_str = latex_str.replace("False", r'\FalseMarker')
    
    latex_lines = [
        re.sub(r'\s+', ' ', line)
        for line in latex_str.split(r"\\")
    ]
    latex_str = "\\\\ \n".join(latex_lines)
    return f'\\begin{{table}}[h]\n\\centering\n\\caption{{{caption}}}\n{latex_str}\\end{{table}}'

In [96]:
_hpr = hpr.loc[MODELS_ORDER, HPR_COLS].rename(index=LABELS).reset_index()

In [97]:
print(df_to_latex_table(_hpr.copy(), "NequIP hyperparameters"))

\begin{table}[h]
\centering
\caption{NequIP hyperparameters}
\begin{tabular}{lrllll} \hline \hline index & num\_layers & optimizer\_amsgrad & use\_ema & BesselBasis\_trainable & rescaled \\ 
 \hline no rescaling & 2 & \TrueMarker & \TrueMarker & \TrueMarker & \FalseMarker \\ 
 rescaling & 2 & \TrueMarker & \TrueMarker & \FalseMarker & \TrueMarker \\ 
 rescaling + bessel & 2 & \TrueMarker & \TrueMarker & \TrueMarker & \TrueMarker \\ 
 2-layer, baseline & 2 & \FalseMarker & \FalseMarker & \TrueMarker & \TrueMarker \\ 
 2-layer, AMSGrad-only & 2 & \TrueMarker & \FalseMarker & \TrueMarker & \TrueMarker \\ 
 2-layer, EMA-only & 2 & \FalseMarker & \TrueMarker & \TrueMarker & \TrueMarker \\ 
 5-layer, baseline & 5 & \FalseMarker & \FalseMarker & \FalseMarker & \TrueMarker \\ 
 5-layer, AMSGrad-only & 5 & \TrueMarker & \FalseMarker & \FalseMarker & \TrueMarker \\ 
 5-layer, EMA-only & 5 & \FalseMarker & \TrueMarker & \FalseMarker & \TrueMarker \\ 
 2-layer & 2 & \TrueMarker & \TrueMarker & \Fa

/tmp/ipykernel_313585/4095877314.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = df.to_latex(index=False)


## Table with the entropy results

In [99]:
ENTROPY_COLS = ["Se", "Sf", "S", "explosion_mean", "explosion_std"]

In [115]:
_sdf = sdf.loc[MODELS_ORDER, ENTROPY_COLS]

_failure = [
    f"${row['explosion_mean'] / 1000:0.2f} \\pm ${row['explosion_std'] / 1000:0.2f}"
    for _, row in _sdf.iterrows()
]
_sdf["Time to failure (ps)"] = _failure
_sdf = _sdf.drop(["explosion_mean", "explosion_std"], axis=1)
_sdf = _sdf.reset_index()

In [117]:
print(df_to_latex_table(_sdf.copy(), "NequIP entropy"))

\begin{table}[h]
\centering
\caption{NequIP entropy}
\begin{tabular}{lrrrl} \hline \hline index & Se & Sf & S & Time to failure (ps) \\ 
 \hline baseline & -1.53 & -0.67 & -0.84 & $1.65 \pm $0.38 \\ 
 non_trainable_bessel & 0.33 & 1.78 & 1.49 & $2.34 \pm $1.27 \\ 
 rescaling & 0.26 & 1.90 & 1.57 & $2.84 \pm $1.26 \\ 
 2-layer, baseline & 0.13 & 1.98 & 1.61 & $2.04 \pm $0.76 \\ 
 2-layer, AMSGrad-only & 0.35 & 1.78 & 1.50 & $2.24 \pm $1.14 \\ 
 2-layer, EMA-only & -0.13 & 1.87 & 1.47 & $1.51 \pm $0.45 \\ 
 5-layer, baseline & 2.02 & 2.13 & 2.11 & $2.97 \pm $1.54 \\ 
 5-layer, AMSGrad-only & 2.29 & 2.35 & 2.34 & $3.62 \pm $1.65 \\ 
 5-layer, EMA-only & 1.41 & 2.16 & 2.01 & $2.27 \pm $1.03 \\ 
 2-layer & 0.33 & 1.80 & 1.51 & $1.96 \pm $0.59 \\ 
 3-layer & 0.09 & 2.34 & 1.89 & $3.64 \pm $1.82 \\ 
 4-layer & 0.19 & 2.48 & 2.02 & $3.67 \pm $1.64 \\ 
 5-layer & 2.14 & 2.31 & 2.28 & $4.62 \pm $1.63 \\ 
 \hline \hline \end{tabular} \end{table}


/tmp/ipykernel_313585/1041434108.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = df.to_latex(index=False, float_format="%.2f", escape=False)


## Table with the extrapolation errors

## 3BPA, energies

In [42]:
EXT_COLS = ["T", 25, 125, 250, 500]
EXT_ORDER = MODELS["2-OPT"] + MODELS["5-OPT"] + MODELS["LAYERS"]

_ext = ext.loc[ext.nsamples > 5]

_exts = []

for T in [300, 600, 1200]:
    vals = f"test_{T}K_energy"
    _tmp = _ext.pivot(index="label", columns="nsamples", values=vals)
    _tmp = _tmp * 1000
    _tmp["T"] = T
    _exts.append(_tmp)

_ext_e = pd.concat(_exts)
_ext_e = _ext_e.sort_values(["label", "T"])[EXT_COLS].loc[EXT_ORDER].reset_index()

In [43]:
print(df_to_latex_table(_ext_e.copy(), "NequIP Energy RMSE", float_format="%.1f"))

\begin{table}[h]
\centering
\caption{NequIP Energy RMSE}
\begin{tabular}{lrrrrr} \hline \hline label & T & 25 & 125 & 250 & 500 \\ 
 \hline 2-layer, baseline & 300 & 2.7 & 1.0 & 1.3 & 1.6 \\ 
 2-layer, baseline & 600 & 3.3 & 1.7 & 1.7 & 1.8 \\ 
 2-layer, baseline & 1200 & 5.8 & 3.8 & 3.7 & 3.7 \\ 
 2-layer, AMSGrad-only & 300 & 3.5 & 1.1 & 0.7 & 0.3 \\ 
 2-layer, AMSGrad-only & 600 & 3.6 & 1.7 & 1.2 & 0.8 \\ 
 2-layer, AMSGrad-only & 1200 & 5.4 & 3.5 & 2.8 & 2.5 \\ 
 2-layer, EMA-only & 300 & 2.7 & 1.0 & 0.5 & 3.7 \\ 
 2-layer, EMA-only & 600 & 3.4 & 1.7 & 1.1 & 3.7 \\ 
 2-layer, EMA-only & 1200 & 6.0 & 3.8 & 3.3 & 5.6 \\ 
 5-layer, baseline & 300 & 2.0 & 2.8 & 8.3 & 9.4 \\ 
 5-layer, baseline & 600 & 2.9 & 2.8 & 8.1 & 9.2 \\ 
 5-layer, baseline & 1200 & 6.0 & 3.7 & 8.9 & 9.9 \\ 
 5-layer, AMSGrad-only & 300 & 2.1 & 0.5 & 1.5 & 0.3 \\ 
 5-layer, AMSGrad-only & 600 & 3.1 & 1.0 & 1.6 & 0.6 \\ 
 5-layer, AMSGrad-only & 1200 & 6.6 & 2.1 & 2.8 & 1.9 \\ 
 5-layer, EMA-only & 300 & 1.9 & 1.1 

/tmp/ipykernel_108955/2674757401.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = df.to_latex(index=False, escape=False, **kwargs)


## 3BPA, forces

In [44]:
EXT_COLS = ["T", 25, 125, 250, 500]
EXT_ORDER = MODELS["2-OPT"] + MODELS["5-OPT"] + MODELS["LAYERS"]

_ext = ext.loc[ext.nsamples > 5]

_exts = []

for T in [300, 600, 1200]:
    vals = f"test_{T}K_forces"
    _tmp = _ext.pivot(index="label", columns="nsamples", values=vals)
    _tmp = _tmp * 1000
    _tmp["T"] = T
    _exts.append(_tmp)

_ext_f = pd.concat(_exts)
_ext_f = _ext_f.sort_values(["label", "T"])[EXT_COLS].loc[EXT_ORDER].reset_index()

In [45]:
print(df_to_latex_table(_ext_f.copy(), "NequIP Forces RMSE", float_format="%.1f"))

\begin{table}[h]
\centering
\caption{NequIP Forces RMSE}
\begin{tabular}{lrrrrr} \hline \hline label & T & 25 & 125 & 250 & 500 \\ 
 \hline 2-layer, baseline & 300 & 92.1 & 45.9 & 34.0 & 25.7 \\ 
 2-layer, baseline & 600 & 140.8 & 86.5 & 68.2 & 54.4 \\ 
 2-layer, baseline & 1200 & 248.0 & 181.6 & 161.8 & 146.3 \\ 
 2-layer, AMSGrad-only & 300 & 91.7 & 45.0 & 33.6 & 25.0 \\ 
 2-layer, AMSGrad-only & 600 & 139.2 & 83.8 & 67.5 & 52.5 \\ 
 2-layer, AMSGrad-only & 1200 & 248.6 & 173.4 & 154.0 & 132.9 \\ 
 2-layer, EMA-only & 300 & 93.2 & 45.6 & 34.0 & 25.4 \\ 
 2-layer, EMA-only & 600 & 143.2 & 86.3 & 68.3 & 54.1 \\ 
 2-layer, EMA-only & 1200 & 251.0 & 181.4 & 168.6 & 151.8 \\ 
 5-layer, baseline & 300 & 88.7 & 30.0 & 20.4 & 16.1 \\ 
 5-layer, baseline & 600 & 133.8 & 57.3 & 43.0 & 34.9 \\ 
 5-layer, baseline & 1200 & 233.4 & 129.0 & 115.0 & 98.8 \\ 
 5-layer, AMSGrad-only & 300 & 87.8 & 28.6 & 19.4 & 14.4 \\ 
 5-layer, AMSGrad-only & 600 & 134.1 & 55.0 & 40.8 & 31.2 \\ 
 5-layer, AMSGrad-o

/tmp/ipykernel_108955/2674757401.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = df.to_latex(index=False, escape=False, **kwargs)


In [50]:
_ext = pd.concat([
    _ext_e.set_index(["label", "T"]),
    _ext_f.set_index(["label", "T"]),
], axis=1).reset_index()

In [52]:
print(df_to_latex_table(_ext.copy(), "NequIP RMSE", float_format="%.1f"))

\begin{table}[h]
\centering
\caption{NequIP RMSE}
\begin{tabular}{lrrrrrrrrr} \hline \hline label & T & 25 & 125 & 250 & 500 & 25 & 125 & 250 & 500 \\ 
 \hline 2-layer, baseline & 300 & 2.7 & 1.0 & 1.3 & 1.6 & 92.1 & 45.9 & 34.0 & 25.7 \\ 
 2-layer, baseline & 600 & 3.3 & 1.7 & 1.7 & 1.8 & 140.8 & 86.5 & 68.2 & 54.4 \\ 
 2-layer, baseline & 1200 & 5.8 & 3.8 & 3.7 & 3.7 & 248.0 & 181.6 & 161.8 & 146.3 \\ 
 2-layer, AMSGrad-only & 300 & 3.5 & 1.1 & 0.7 & 0.3 & 91.7 & 45.0 & 33.6 & 25.0 \\ 
 2-layer, AMSGrad-only & 600 & 3.6 & 1.7 & 1.2 & 0.8 & 139.2 & 83.8 & 67.5 & 52.5 \\ 
 2-layer, AMSGrad-only & 1200 & 5.4 & 3.5 & 2.8 & 2.5 & 248.6 & 173.4 & 154.0 & 132.9 \\ 
 2-layer, EMA-only & 300 & 2.7 & 1.0 & 0.5 & 3.7 & 93.2 & 45.6 & 34.0 & 25.4 \\ 
 2-layer, EMA-only & 600 & 3.4 & 1.7 & 1.1 & 3.7 & 143.2 & 86.3 & 68.3 & 54.1 \\ 
 2-layer, EMA-only & 1200 & 6.0 & 3.8 & 3.3 & 5.6 & 251.0 & 181.4 & 168.6 & 151.8 \\ 
 5-layer, baseline & 300 & 2.0 & 2.8 & 8.3 & 9.4 & 88.7 & 30.0 & 20.4 & 16.1 \\ 
 

/tmp/ipykernel_108955/2674757401.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = df.to_latex(index=False, escape=False, **kwargs)
